In [2]:
%load_ext autoreload
%autoreload 2

from deltas.pipeline import data, classifier, evaluation
from deltas.model import base, downsample
import deltas.plotting.plots as plots
from tqdm import tqdm

import numpy as np
import pandas as pd
# np.random.seed(10)


In [3]:

N1 = 1000
N2 = 10
m = 1
v = 1
costs = (1, 1)  # change for (1, 10) to increase results
# Gaussian (not always seperable)
dfs = []
for i in tqdm(range(10)):
    data_clf = data.get_data(
        m1=[-m, -m],
        m2=[m, m],
        cov1=[[v, 0], [0, v]],
        cov2=[[v, 0], [0, v]],
        N1=N1,
        N2=N2,
        scale=False
    )

    model = 'SVM-linear'
    model = 'SVM-rbf'
    # model = 'Linear'
    model = 'MLP'

    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    deltas_model = downsample.downsample_deltas(clf).fit(X, y,
                                                         alpha=10,
                                                         _print=False,
                                                         _plot=False,
                                                         method='supports-prop-update_mean',
                                                         max_trials=10000,
                                                         parallel=True)

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=False, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')

 80%|████████  | 8/10 [01:08<00:15,  7.68s/it]

not fit deltas


100%|██████████| 10/10 [01:28<00:00,  8.87s/it]


In [4]:
print(len(dfs))

9
                  Accuracy    G-Mean  ROC-AUC        F1
Method                                                 
Baseline           0.74035  0.693960  0.74035  0.650091
SMOTE              0.86840  0.865534  0.86840  0.858419
Balanced Weights   0.74975  0.709433  0.74975  0.669615
BMR                0.82890  0.813468  0.82890  0.796503
Threshold          0.82890  0.813468  0.82890  0.796503
Our Method         0.89700  0.895409  0.89700  0.891190
Baseline           0.58245  0.407207  0.58245  0.284466
SMOTE              0.73580  0.709846  0.73580  0.672330
Balanced Weights   0.72630  0.677243  0.72630  0.628932
BMR                0.83245  0.823350  0.83245  0.809005
Threshold          0.83245  0.823350  0.83245  0.809005
Our Method         0.86825  0.867530  0.86825  0.863422
Baseline           0.76065  0.723162  0.76065  0.686776
SMOTE              0.87005  0.865252  0.87005  0.857001
Balanced Weights   0.77985  0.751057  0.77985  0.721347
BMR                0.90815  0.907890  0.90815 

In [7]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
print(mean_df)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)

m = mean_df.to_dict('list')
s = std_df.to_dict('list')
metrics = mean_df.columns.to_list()
methods = mean_df.index.to_list()
sf = 5
for metric in metrics:
    means = m[metric]
    sts = s[metric]
    mx = np.argmax(means)
    for i in range(len(means)):
        m_str = str(means[i])[1:sf]
        if i == mx:
            m_str = f"\\textbf{{{m_str}}}"
        s_str = str(sts[i])[1:sf-1]
        m[metric][i] = f'${m_str} \\pm {s_str}$'

method_map = {
    'Baseline': 'Baseline',
    'SMOTE': "SMOTE \cite{Chawla_2002_JAIR}",
    'Balanced Weights': 'BW',
    'BMR': 'BMR \cite{Bahnsen_2014_SIAM}',
    'Threshold': 'Thresh \cite{Sheng_2006_AAAI}',
    'Our Method': 'Our Method',
}
meths_new = []
for me in methods:
    meths_new.append(method_map[me])
m['Methods'] = meths_new
df = pd.DataFrame(m)  # .set_index('Methods')
meths = df.pop('Methods')
df.insert(0, 'Methods', meths)
latex_str = df.to_latex(index=False)

print('\\begin{tabular}{@{}lrrrr@{}}'+latex_str[22:])

                  Accuracy    G-Mean   ROC-AUC        F1
Baseline          0.668972  0.564094  0.668972  0.484157
SMOTE             0.818933  0.804475  0.818933  0.785487
Balanced Weights  0.693444  0.618180  0.693444  0.552460
BMR               0.864411  0.854531  0.864411  0.843909
Threshold         0.864561  0.854711  0.864561  0.844114
Our Method        0.885144  0.881311  0.885144  0.875657
\begin{tabular}{@{}lrrrr@{}}
\toprule
Methods & Accuracy & G-Mean & ROC-AUC & F1 \\
\midrule
Baseline & $.668 \pm .08$ & $.564 \pm .15$ & $.668 \pm .08$ & $.484 \pm .19$ \\
SMOTE \cite{Chawla_2002_JAIR} & $.818 \pm .05$ & $.804 \pm .06$ & $.818 \pm .05$ & $.785 \pm .07$ \\
BW & $.693 \pm .06$ & $.618 \pm .10$ & $.693 \pm .06$ & $.552 \pm .13$ \\
BMR \cite{Bahnsen_2014_SIAM} & $.864 \pm .06$ & $.854 \pm .07$ & $.864 \pm .06$ & $.843 \pm .09$ \\
Thresh \cite{Sheng_2006_AAAI} & $.864 \pm .06$ & $.854 \pm .07$ & $.864 \pm .06$ & $.844 \pm .09$ \\
Our Method & $\textbf{.885} \pm .02$ & $\textbf{.881

In [6]:
df.loc['BMR'].mean().to_list()
df.columns.to_list()

KeyError: 'BMR'